In [1]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras

In [2]:
print(tf.config.experimental.list_physical_devices())
print(tf.test.is_built_with_cuda())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True


In [3]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='D:\Github\Learning-dl\data_flowers', untar=True)

In [4]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('/tmp/.keras/datasets/flower_photos')

In [5]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [6]:
X,y = [],[]
for flowers_names,class_img in flowers_labels_dict.items():
    images_paths = list(data_dir.glob(f'{flowers_names}/*.jpg'))
    for path in images_paths:
        img = cv2.imread(str(path))
        resize_img = cv2.resize(img,(224,224))
        X.append(resize_img)
        y.append(class_img)

In [7]:
X = np.array(X)
y = np.array(y)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

In [9]:
X_train = X_train/255
X_test = X_test/255

In [10]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [11]:
num_of_flowers = 5
model = keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [12]:
with tf.device('/GPU:0'):
  model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])

  model.fit(X_train, y_train, epochs=5)

Epoch 1/5
86/86 [==============================] - 15s 40ms/step - loss: 0.8320 - acc: 0.6919
Epoch 2/5
86/86 [==============================] - 3s 39ms/step - loss: 0.4405 - acc: 0.8492
Epoch 3/5
86/86 [==============================] - 4s 42ms/step - loss: 0.3469 - acc: 0.8805
Epoch 4/5
86/86 [==============================] - 4s 45ms/step - loss: 0.2936 - acc: 0.9041
Epoch 5/5
86/86 [==============================] - 3s 40ms/step - loss: 0.2513 - acc: 0.9233


In [13]:
model.evaluate(X_test,y_test)

29/29 [==============================] - 2s 57ms/step - loss: 0.2897 - acc: 0.8900


[0.2896856367588043, 0.8899782299995422]